<a target="_blank" href="https://colab.research.google.com/github/Sebelino/DD2424-project/blob/main/e1_sebastian.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [4]:
%load_ext autoreload
%autoreload 2

import os
import socket

repo_name = "DD2424-project"
dep_filename = "download_dataset.py"
cwd = os.getcwd()
hostname = socket.gethostname()

print(f"Host: {hostname}")
print(f"You are here: {cwd}")

if os.path.exists(dep_filename):
    print(f"{dep_filename} is in your current directory. Assuming you are in the root of the repository.")
elif not os.path.exists(repo_name):
    print(f"Cloning repo and fetching {dep_filename}...")
    !git clone https://github.com/Sebelino/DD2424-project
    !cp DD2424-project/download_dataset.py .
else:
    !git -C DD2424-project pull --rebase
    !cp DD2424-project/download_dataset.py .

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Host: deeplearning-vm
You are here: /home/sebelino/pycharm_deployment
download_dataset.py is in your current directory. Assuming you are in the root of the repository.


# Train model

In [ ]:

from dataclasses import asdict
from util import json_dumps
from evaluation import evaluate_test_accuracy_and_misclassified, \
    evaluate_with_train_val_plot, run_with_different_seeds
from run import run
%reload_ext autoreload
%autoreload 2

from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import

from training import TrainParams, NagParams

training_params = TrainParams(
    seed=determinism.seed,
    batch_size=32,
    architecture="resnet50",
    n_epochs=10,
    optimizer=NagParams(
        learning_rate=0.01,
        weight_decay=1e-4,
        momentum=0.9,
    ),
    freeze_layers=True,
    unfreezing_epochs=(3, 6),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
)

invalidate_cache_entry = False
if invalidate_cache_entry:
    run.call_and_shelve(training_params).clear()

print("Training with params:")
print(json_dumps(training_params.minimal_dict()))
result = run(training_params)

print("Training result:")
print(json_dumps(asdict(result)))
evaluate_with_train_val_plot(result)

run_with_different_seeds(training_params, count=3)

Training with params:
{
    "architecture": "resnet50",
    "batch_size": 32,
    "freeze_layers": true,
    "n_epochs": 10,
    "optimizer": {
        "learning_rate": 0.01,
        "momentum": 0.9,
        "name": "nag",
        "weight_decay": 0.0001
    },
    "seed": 42,
    "unfreezing_epochs": [
        3,
        6
    ],
    "validation_freq": 1
}
Training result:
{
    "epochs": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10
    ],
    "training_accuracies": [
        0.7146739130434783,
        0.923233695652174,
        0.931046195652174,
        0.9847146739130435,
        0.9925271739130435,
        0.9942255434782609,
        0.9962635869565217,
        0.9993206521739131,
        0.9989809782608695,
        0.9989809782608695
    ],
    "training_elapsed": 145.26918099399882,
    "training_losses": [
        1.6689337611846302,
        0.47329614408638165,
        0.25059353301058646,
        0.07627233775

# Test model

In [6]:
# Currently broken

from training import Trainer
from torch.utils.data import DataLoader
from datasets import load_dataset

test_dataset = load_dataset("test", Trainer.make_transform(training_params))
test_loader = DataLoader(
    test_dataset,
    batch_size=training_params.batch_size,
    shuffle=False,
    num_workers=2,
    worker_init_fn=Determinism.data_loader_worker_init_fn(training_params.seed),
)
print(f"Test size: {len(test_loader.dataset)}")

evaluate_test_accuracy_and_misclassified(result, test_loader, test_dataset)


Test size: 3669


AttributeError: 'TrainingResult' object has no attribute 'trainer'